## Concepts and Best practice

xdaq web site: https://svnweb.cern.ch/trac/cmsos/wiki

### XDAQ Concepts 

#### Xdaq Executive

A process containing multiple functional units. These units are loaded to the executive at runtime as plugins. What should be loaded into the executive are configured by the user configuration xml file and a common profile xml file jointly. 

An example non-trivial user configuration xml:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<xc:Partition xmlns:soapenc="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xc="http://xdaq.web.cern.ch/xdaq/xsd/2004/XMLConfiguration-30" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">	
	<xc:Context url="http://srv-s2d16-29-01.cms:6868">
	  <xc:Endpoint protocol="utcp" service="b2in" rmode="select" hostname="srv-s2d16-29-01.cms" port="6867" network="utcp1" sndTimeout="2000" rcvTimeout="2000" affinity="RCV:P,SND:W,DSR:W,DSS:W" singleThread="true" publish="true"/>   
	
	<xc:Application class="eventing::core::Subscriber" id="12" instance="1" network="utcp1" group="b2in" service="eventing-subscriber" bus="brildata">
	  <properties xmlns="urn:xdaq-application:eventing::core::Subscriber" xsi:type="soapenc:Struct">
	    <eventings xsi:type="soapenc:Array" soapenc:arrayType="xsd:ur-type[1]">
	      <item xsi:type="xsd:string" soapenc:position="[0]">utcp://srv-s2d16-32-01.cms:8181</item>      
	    </eventings>
	  </properties>
	</xc:Application>
	<xc:Module>${XDAQ_ROOT}/lib/libb2inutils.so</xc:Module>
    <xc:Module>${XDAQ_ROOT}/lib/libeventingcore.so</xc:Module>   
	
	<xc:Application class="bril::supervisor::LuminosityMonitor"  id="101" instance="0" network="local" service="detluminosity" logpolicy="inherit">
	  <properties xmlns="urn:xdaq-application:bril::supervisor::LuminosityMonitor" xsi:type="soapenc:Struct">
	    <bus xsi:type="xsd:string">brildata</bus>
	    <lumitopics xsi:type="xsd:string">pltlumizero,bcm1flumi</lumitopics>
	  </properties>
	</xc:Application>
	<xc:Module>${XDAQ_ROOT}/lib/libeventingapi.so</xc:Module>
    <xc:Module>${XDAQ_ROOT}/lib/libbrilsupervisor.so</xc:Module>
  </xc:Context>	
</xc:Partition>
```

Example profile:
```
more /opt/xdaq/etc/default.profile
```

An executive can be started from commandline /opt/xdaq/bin/xdaq.exe. Where option '-e' takes the profile to load , '-c' takes the user configuration. Instead of '-e', one can use '-z bril' which means to take the profile default to the BRIL cluster. If -z bril is used, pt::utcp::Application configuration block must not be in the user configuration because it is already defined in the BRIL profile. 

Note: only absolute files paths are recognized by command options taking file names.

( See XDAQ developer manual chapter 2. )


#### InfoSpace and xdata

As an XDAQ executive consists of many applications, InfoSpace is a common space for them to exchange data. Only a specific data category xdata can stay in the InfoSpace. There are events, event callbacks associated with the status of xdata variables in the InfoSpace. Any number of InfoSpace can be created by the user application, but there is a special one "application infospace" which is shared with all applications in the same process. 

InfoSpace, infospace callbacks and xdata are the fundamental blocks for building the application configuration and monitoring infrastructure. 

( See XDAQ developer manual Chapter 12 )

#### Event , Event call-back and Event-Driven programming

XDAQ is event-driven. The flow of the program is determined by events: something happens (Event Call-Back) only when something else (Event) happens. The main thread listens for events, then triggers callback functions when one of those events is detected. Since timing and synchronisation is vital to online applications, event-driven programming ensures nothing blocks or delays the flow of the main thread (asynchronous execution). 

There are two types of Events : xdata::Event, toolbox::Event 

Their corresponding interface class are xdata::ActionListener, toolbox::ActionListener.

Their call-back and event-listening functions are:

xdata::ActionListener::actionPerformed(xdata::Event&),  xdata::InfoSpace::addListener toolbox::ActionListener::actionPerformed(toolbox::Event&), toolbox::ActionListener::addActionListener


The user application can fire its own toolbox::Event being a toolbox::EventDispatcher. Example user-defined events see bril/vdmmonitor/Events.h


#### Workloop, timer and multithread programming

There are many reasons to use multiple threads in an online application. 

A typical scenario is: as a reaction to an event, you start processing a lot of data, if the processing is done in the main thread it could delay the program flow since it takes so long time. In this case, you should move the data processing to a new thread, i.e. launch a xdaq workloop. 

When you need to periodically check something, e.g. a cache size, or do something at fixed time, e.g. 2 minutes after the constructor is created, you can launch a xdaq timer, which is also a new thread. 


( XDAQ developer manual Chapter 4 )

#### A xdaq executive is a web server

A xdaq executive corresponds to a web-site and you can control the content of the site from the user application via CGI programming. 

( XDAQ developer manual Chapter 14 , CGICC programming https://www.gnu.org/software/cgicc/index.html)

#### Publisher/Subscriber model (Eventing) for interprocess message exchange

Eventing is a publisher and subscriber model for exchanging messages between xdaq processes (executives). Publishers and subscribers do not know each other, they exchange data via an intermediary event bus managed by central servers.

( XDAQ developer manual Chapter 19 )

### BRIL DAQ specifics

#### BRIL daq is a collection of pure xdaq applications

We choose not to build any other common frameworks on top of the xdaq framework. The only dependency between brildaq components is the data definition for the interprocess communication. 

Since bril daq applications use only xdaq framework and nothing else, it requires the developer to have a very good understanding of the framework and to use it well. The benefit of the loosely coupled design of brildaq is the very simple software management even though the system contains many detectors.

#### BRIL daq uses subscriber/publisher model for interprocess communication

We choose to use publisher/subscriber model (b2in-eventing) for interprocess communication. The format of the messages in the BRIL data flow are predefined and agreed by the subscribers and publishers.

All message formats are defined in /daq/interface/bril. 



### Best Practice

#### Do not use web browser on .cms hosts, use local browsers and set up ssh tunnel to access online applications. https://twiki.cern.ch/twiki/bin/view/CMS/BrilDAQP5Tunnel

### Use namespaces

#### When developing, use developer hosts, e.g. brildev1, instead of the production hosts.

#### When developing, run outside the BRIL zone

#### When testing eventing, test against your own eventing server or the mirror server

#### Prefer utility classes from xdaq framework over classes from native language or external libraries, e.g. semaphor, thread

#### Use your own home directory for private data and code, /globalscratch for temporary code and data. /brildata and /brilpro are for BRIL common data staging and common code staging purpose. 
    
    Always login as user brilpro to write into /brildata and /brilpro
    